In [1]:
# import necessary libraries

import face_recognition as fr
import os
import numpy as np
import cv2 as cv

In [2]:
# capture video from default webcam

video = cv.VideoCapture(0)

In [3]:
ROOT = 'Desktop\\datasets\\avengers_images\\train'

train_images = []
train_labels = []

for file in os.listdir(ROOT):
    path = os.path.join(ROOT, file)
    for image in os.listdir(path):
        img_path = os.path.join(path, image)
        img = fr.load_image_file(img_path)
        img_encoding = fr.face_encodings(img)
        try:
            train_images.append(img_encoding[0])
            train_labels.append(file)
        except:
            continue
        
print(len(train_images))
print(len(train_labels))

251
251


In [4]:
while True:
    # Grab a single frame of video
    ret, frame = video.read()

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    # Find all the faces and face encodings in the current frame of video
    face_locations = fr.face_locations(frame)
    face_encodings = fr.face_encodings(frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        matches = fr.compare_faces(train_images, face_encoding)
        name = 'No match found'

        # Use the known face with the smallest distance to the new face
        face_distances = fr.face_distance(train_images, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = train_labels[best_match_index]

        face_names.append(name)


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Draw a box around the face
        cv.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), thickness=1)

        # Draw a label with a name below the face
        cv.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), thickness=-1)
        font = cv.FONT_HERSHEY_DUPLEX
        cv.putText(frame, name, (left + 6, bottom - 6), font, fontScale=1, (255, 255, 255), thickness=1)

    # Display the resulting image
    cv.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv.waitKey(1) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break
        
# Release handle to the webcam
video.release()